In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint


In [6]:
DatadirectoryTrain= "Training1/train/"
DatadirectoryTest= "Training1/test/"

In [7]:
Emotions=["0","1","2","3","4","5","6"]

In [8]:
trainingData=[]
img_size=224
def createTrainingData():
    for category in Emotions:
        path=os.path.join(DatadirectoryTrain,category)
        emotion_num=Emotions.index(category) # emotiion numbers are the labels
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                temp_Array = cv2.resize(img_array,(img_size,img_size))
                trainingData.append([temp_Array,emotion_num])
            except Exception as e:
                pass  
            
testData=[]
def createTestData():
    for category in Emotions:
        path=os.path.join(DatadirectoryTest,category)
        emotion_num=Emotions.index(category) # emotiion numbers are the labels
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                temp_Array = cv2.resize(img_array,(img_size,img_size))
                testData.append([temp_Array,emotion_num])
            except Exception as e:
                pass  

In [9]:
createTrainingData()
#createTestData()

In [10]:
print(len(trainingData))
#print(len(testData))

1425


In [11]:

random.shuffle(trainingData) # should not learn the sequence so shuffle


In [12]:
X = [] #DATA
Y = [] #LABEL
for features,label in trainingData:
    X.append(features)
    Y.append(label)
    
X = np.array(X).reshape(-1,img_size,img_size,3)

X_test = [] #DATA
Y_test = [] #LABEL
for features,label in testData:
    X_test.append(features)
    Y_test.append(label)
    
X_test = np.array(X_test).reshape(-1,img_size,img_size,3)

In [13]:
#normalize data
X=X/255.0
X_test=X_test/255.0


In [14]:
Y=np.array(Y)
#Y_test=np.array(Y_test)

# DEEP LEARNING MODEL FOR TRAINING - TRANSFER LEARNING

In [15]:
model = tf.keras.applications.MobileNetV2() ##pre trained model


# Transfer learning - tuning,weight will start from last check point

In [16]:
base_input=model.layers[0].input

In [17]:
base_output=model.layers[-2].output

In [18]:
finalOutput=layers.Dense(128)(base_output) #adding new layer after the output of global pooling layer
finalOutput=layers.Activation('relu')(finalOutput)#activation func
finalOutput=layers.Dense(64)(finalOutput) 
finalOutput=layers.Activation('relu')(finalOutput)
finalOutput=layers.Dense(7,activation='softmax')(finalOutput)#there are 7 emotions, classification layer

In [19]:
newModel = keras.Model(inputs=base_input,outputs=finalOutput)


In [20]:
newModel.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [21]:
newModel.compile(loss="sparse_categorical_crossentropy",optimizer="adam", metrics=["accuracy"])

In [ ]:
#H=newModel.fit(X,Y,epochs=25)
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
H=newModel.fit(X,Y,epochs=20,callbacks=[checkpoint],validation_split=0.2)


Epoch 1/20
36/36 [==============================] - 132s 3s/step - loss: 1.7939 - accuracy: 0.3132 - val_loss: 2.8616 - val_accuracy: 0.1509


C:\Users\Burcu\anaconda3\envs\python37\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: model-001.model\assets
Epoch 2/20
36/36 [==============================] - 129s 4s/step - loss: 1.4213 - accuracy: 0.4789 - val_loss: 6.3122 - val_accuracy: 0.1509
Epoch 3/20
36/36 [==============================] - 127s 4s/step - loss: 1.1696 - accuracy: 0.5789 - val_loss: 9.1959 - val_accuracy: 0.1474
Epoch 4/20
36/36 [==============================] - 126s 3s/step - loss: 0.8740 - accuracy: 0.6798 - val_loss: 3.5992 - val_accuracy: 0.1544
Epoch 5/20
36/36 [==============================] - 131s 4s/step - loss: 0.7889 - accuracy: 0.7228 - val_loss: 2.7457 - val_accuracy: 0.2246


C:\Users\Burcu\anaconda3\envs\python37\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: model-005.model\assets
Epoch 6/20
36/36 [==============================] - 132s 4s/step - loss: 0.6449 - accuracy: 0.7921 - val_loss: 5.3723 - val_accuracy: 0.1860
Epoch 7/20
36/36 [==============================] - 138s 4s/step - loss: 0.5161 - accuracy: 0.8175 - val_loss: 4.3489 - val_accuracy: 0.2140
Epoch 8/20
36/36 [==============================] - 131s 4s/step - loss: 0.3637 - accuracy: 0.8763 - val_loss: 5.9237 - val_accuracy: 0.1439
Epoch 9/20
27/36 [=====================>........] - ETA: 30s - loss: 0.3754 - accuracy: 0.8588

In [ ]:
newModel.evaluate

In [ ]:
newModel.save('myModel-v3.h5')


In [5]:
N = 20
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

NameError: name 'H' is not defined

<Figure size 432x288 with 0 Axes>

## USE LEARNING MODEL 

In [2]:
newModel=tf.keras.models.load_model('myModel-v2.h5')

In [4]:
#WITH WEBCAM

path="haarcascade_frontalface_default.xml"
fontScale=1.5
font=cv2.FONT_HERSHEY_PLAIN

rectangle_bgr=(255,255,255)
img=np.zeros((500,500))

message="Msj on the box"
(messageWidth,messageHeight)=cv2.getTextSize(message,font,fontScale=fontScale,thickness=1)[0]

msgOffsetX=10
msgOffsetY=img.shape[0]-25

box_coords=((msgOffsetX,msgOffsetY),(msgOffsetX+messageWidth+2,msgOffsetY-messageHeight-2))
cv2.rectangle(img,box_coords[0],box_coords[1],rectangle_bgr,cv2.FILLED)
cv2.putText(img,message,(msgOffsetX,msgOffsetY),font,fontScale=fontScale,color=(0,0,0),thickness=1)

cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
    
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    faces=faceCascade.detectMultiScale(gray,1.1,4)
    for x,y,w,h in faces:
        roi_gray=gray[y:y+h,x:x+w]
        roi_color=frame[y:y+h,x:x+w]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2) #BGR
        facess=faceCascade.detectMultiScale(roi_gray)
       
        for(ex,ey,ew,eh) in facess:
            face_roi=roi_color[ey:ey+eh,ex:ex+ew]

    finalImg=cv2.resize(face_roi,(224,224))
    finalImg=np.expand_dims(finalImg,axis=0)
    finalImg=finalImg/255.0
    
    font=cv2.FONT_HERSHEY_SIMPLEX
    Predictions=newModel.predict(finalImg)
    
    fontScale=1.5
    font=cv2.FONT_HERSHEY_PLAIN
        
    if(np.argmax(Predictions)==0):
        status="Angry"
        
        x1,y1,w1,h1=0,0,175,75
        
        cv2.rectangle(frame,(x1,x1),(x1+w1,y1+h1),(0,0,0),-1)
        cv2.putText(frame,status,(x1+int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
        cv2.putText(frame,status,(100,150),font,3,(0,0,255),2,cv2.LINE_4)
       #  cv2.rectangle(frame,(x,y),(x+w,x+h),(0,0,255))
        
    elif (np.argmax(Predictions)==1):

        status="Disgust"
        
        x1,y1,w1,h1=0,0,175,75
        
        cv2.rectangle(frame,(x1,x1),(x1+w1,y1+h1),(0,0,0),-1)
        cv2.putText(frame,status,(x1+int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
        cv2.putText(frame,status,(100,150),font,3,(0,0,255),2,cv2.LINE_4)
        # cv2.rectangle(frame,(x,y),(x+w,x+h),(0,0,255))
    
    
    elif (np.argmax(Predictions)==2):

        status="Fear"
        
        x1,y1,w1,h1=0,0,175,75
        
        cv2.rectangle(frame,(x1,x1),(x1+w1,y1+h1),(0,0,0),-1)
        cv2.putText(frame,status,(x1+int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
        cv2.putText(frame,status,(100,150),font,3,(0,0,255),2,cv2.LINE_4)
        # cv2.rectangle(frame,(x,y),(x+w,x+h),(0,0,255))
    
    elif (np.argmax(Predictions)==3):

        status="Happy"
        
        x1,y1,w1,h1=0,0,175,75
        
        cv2.rectangle(frame,(x1,x1),(x1+w1,y1+h1),(0,0,0),-1)
        cv2.putText(frame,status,(x1+int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
        cv2.putText(frame,status,(100,150),font,3,(0,0,255),2,cv2.LINE_4)
       # cv2.rectangle(frame,(x,y),(x+w,x+h),(0,0,255))
        
    elif (np.argmax(Predictions)==4):

        status="Neutral"
        
        x1,y1,w1,h1=0,0,175,75
        
        cv2.rectangle(frame,(x1,x1),(x1+w1,y1+h1),(0,0,0),-1)
        cv2.putText(frame,status,(x1+int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
        cv2.putText(frame,status,(100,150),font,3,(0,0,255),2,cv2.LINE_4)
       #  cv2.rectangle(frame,(x,y),(x+w,x+h),(0,0,255))
    

    elif (np.argmax(Predictions)==5):

        status="Sad"
        
        x1,y1,w1,h1=0,0,175,75
        
        cv2.rectangle(frame,(x1,x1),(x1+w1,y1+h1),(0,0,0),-1)
        cv2.putText(frame,status,(x1+int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
        cv2.putText(frame,status,(100,150),font,3,(0,0,255),2,cv2.LINE_4)
       #  cv2.rectangle(frame,(x,y),(x+w,x+h),(0,0,255))
    
    
    else:
        status="Surprise"
        
        x1,y1,w1,h1=0,0,175,75
        
        cv2.rectangle(frame,(x1,x1),(x1+w1,y1+h1),(0,0,0),-1)
        cv2.putText(frame,status,(x1+int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
        cv2.putText(frame,status,(100,150),font,3,(0,0,255),2,cv2.LINE_4)
        # cv2.rectangle(frame,(x,y),(x+w,x+h),(0,0,255))
    
    cv2.imshow('Input', frame)

    c = cv2.waitKey(1)
    if c == 27:
        break

cap.release()
cv2.destroyAllWindows()